# tools

In [ ]:
tool_response.conversation

In [ ]:

from components.etl.chat_prompt import ChatPrompt


prompt = ChatPrompt(    
    name="completion",    
)

In [ ]:



from components.etl.chat_agent import ChatAgent
from dataclasses import Field
from langchain_core.pydantic_v1 import BaseModel, Field
from components.etl.chat_prompt import ChatPrompt

from langchain_core.utils.function_calling import convert_to_openai_tool

from components.etl.chat_prompt import get_tool_scheduler_prompt

prompt = ChatPrompt(
    "experimental/pirate",
    name="pirate_prompt",    
    )


await prompt(prompt="Hello, how are you?")

# agent

In [ ]:




from components.etl.chat_agent import ChatAgent
from dataclasses import Field
from langchain_core.pydantic_v1 import BaseModel, Field
from components.etl.chat_prompt import ChatPrompt

from langchain_core.utils.function_calling import convert_to_openai_tool

from components.etl.chat_prompt import get_tool_scheduler_prompt

prompt = ChatPrompt(
    "experimental/pirate",
    name="pirate_prompt",    
    )


class SwordTool(BaseModel):
    """this is a sword that the pirate can use to fight his enemies."""
    draw: bool = Field(..., description="true or false if the sword is drawn or not")
    sharp: bool = Field(..., description="true or false if the sword is sharp or not")
    length: float = Field(..., description="the length of the sword")


class GunTool(BaseModel):
    """this is a gun that the pirate can use to fight his enemies."""
    loaded: bool = Field(..., description="true or false if the gun is loaded or not")
    caliber: float = Field(..., description="the caliber of the gun")
    range: float = Field(..., description="the range of the gun")


class OfferMission(BaseModel):
    """this is a mission that the pirate can offer to the other pirates."""
    mission: str = Field(..., description="the mission that the pirate wants to offer to the other pirates")
    reward: float = Field(..., description="the reward that the pirate wants to offer to the other pirates")


class OfferDrink(BaseModel):
    """this is a drink that the pirate can offer to the other pirates."""
    drink: str = Field(..., description="the drink that the pirate wants to offer to the other pirates")
    price: float = Field(..., description="the price that the pirate wants to offer to the other pirates")



# tools = [get_tool_scheduler_prompt(convert_to_openai_tool(tool)) 
# for tool in [SwordTool, GunTool]]


agent = ChatAgent(
    "experimental/pirate_agent", 
    name="pirate_agent",
    tools=[SwordTool, GunTool, OfferMission, OfferDrink],
)


state = {
    "pirate":{
        "health": 100,
        "money": 100,
    },
    "visitor":{
        "health": 100,
        "money": 1000,
        "weapon": "sword",
    }
}

In [ ]:
message = "hello there mate!"

agent_response = await agent.next_step(
    message=message,
    **state
)
# print("pirate:", prompt_response.conversation[-2].content)
# print("action", prompt_response.value)
# assert len(agent.conversation) == 3
agent_response.conversation

In [ ]:
from components.etl.conversation import ConversationRag

conv_rag = ConversationRag("pirate")


In [ ]:
examples = await conv_rag.similarity(agent_response.conversation)
examples[0]

In [ ]:
examples[0]

In [ ]:
message = "I am here to kill you!"

agent_response = await agent.next_step(
    message=message,
    **state
)
# print("pirate:", pirate_output)
# print("action", pirate_action)
agent_response.conversation

In [ ]:
message = "is was a joke! just give me a mission"

agent_response = await agent.next_step(
    message=message,
    **state
)
# print("pirate:", pirate_output)
# print("action", pirate_action)
agent_response.conversation

In [ ]:
await agent.prompt.with_tools(
    # prompt=pirate_output.value,
    # prompt="*SwordTool: {draw: true, sharp: true, length: 'long'}*",
    prompt="""pirate: "Avast ye scurvy dog! Ye be talkin' bold words to a pirate! Draw me sword! Let's see if ye be able to back up yer threats! Arrr! SwordTool: draw: true, sharp: true, length: 32}""",
    tools=agent.tools
)   

# RAG

In [ ]:

from components.etl.conversation import LangsmithConversationDataset


dataset = LangsmithConversationDataset("pirate")

conversation_list = dataset.get_records(is_agent=True)

In [ ]:
conversation_list[0]

In [ ]:
from components.etl.conversation import ConversationRag

conv_rag = ConversationRag("pirate")

In [ ]:
await conv_rag.add_conversation_list(conversation_list)

In [ ]:
examples = await conv_rag.get_many()
examples

In [ ]:
examples[0].metadata

In [ ]:


from components.etl.conversation import RagConversationVectorizer, ConversationRagValue
from components.etl.rag_manager import RagVectorSpace, RagVector

rag_vectorizer = RagConversationVectorizer()
rag_space = RagVectorSpace("pirate", rag_vectorizer, ConversationRagValue)

# await rag_space.get_many()


In [ ]:
res = await rag_space.add_many(conversation_list)
# res = await rag_space.vectorizer.embed_documents(conversation_list)
res

In [ ]:
await rag_space.delete_all()

# test prompt RAG

In [ ]:
from components.etl.chat_prompt import ChatPrompt



test_segmentizer = ChatPrompt(
    "test_prompts/test_segmentizer", 
    name="test_segmentizer", 
    rag_length=3
)

# sentence="queen Elizabeth II of England, had a networth of 1000000$ in 1967, and had 4 children with prince Philip, duke of edinburgh."
# sentence = "the F-16, was invented in the USA in 1974, by General Dynamics."
sentence = "the s&p 500 increased by 10% in Q2 of 2020."
# sentence = "Adolf Hitler, was the leader of germany, during WW2."
# sentence = "Apple is known for the iPhone, iPad, and Macbook."
# sentence = "ООН была создана 24 октября 1945 года."
output = await test_segmentizer(sentence=sentence)

output


In [ ]:
from components.etl.conversation import ConversationRag, LangsmithConversationDataset


segmentizer_rag = ConversationRag("test_segmentizer")
dataset = LangsmithConversationDataset("test_segmentizer")



In [ ]:
conversation_list = dataset.get_records(is_agent=False)
conversation_list

In [ ]:

all_examples = await segmentizer_rag.get_many()

# assert len(all_examples) == 0

examples_list = conversation_list[:1]

add_response = await segmentizer_rag.add_conversation_list(examples_list)

assert len(add_response['records']) == len(examples_list)
assert add_response['records'][0]['id'] == examples_list[0].id


all_examples = await segmentizer_rag.get_many()

assert len(all_examples) == 1


In [ ]:

# all_examples = await segmentizer_rag.get_many()

# # assert len(all_examples) == 0

# examples_list = conversation_list

# add_response = await segmentizer_rag.add_conversation_list(examples_list)
# add_response
# assert len(add_response['records']) == len(examples_list)
# assert add_response['records'][0]['id'] == examples_list[0].id


all_examples = await segmentizer_rag.get_many()

assert len(all_examples) == 2


In [ ]:
from components.etl.chat_prompt import ChatPrompt



test_segmentizer = ChatPrompt(
    "test_prompts/test_segmentizer", 
    name="test_segmentizer", 
    rag_length=3
)

# sentence="queen Elizabeth II of England, had a networth of 1000000$ in 1967, and had 4 children with prince Philip, duke of edinburgh."
# sentence = "the F-16, was invented in the USA in 1974, by General Dynamics."
sentence = "the s&p 500 increased by 10% in Q2 of 2020."
# sentence = "Adolf Hitler, was the leader of germany, during WW2."
# sentence = "Apple is known for the iPhone, iPad, and Macbook."
# sentence = "ООН была создана 24 октября 1945 года."
output = await test_segmentizer(sentence=sentence)

output


In [ ]:
output.examples[0]

In [ ]:
all_examples = await segmentizer_rag.get_many()
all_examples

In [ ]:
await segmentizer_rag.delete_all()

# streaming 

In [ ]:

from components.etl.chat_agent import ChatAgent
from dataclasses import Field
from langchain_core.pydantic_v1 import BaseModel, Field
from components.etl.chat_prompt import ChatPrompt

from langchain_core.utils.function_calling import convert_to_openai_tool

from components.etl.chat_prompt import get_tool_scheduler_prompt, prompt

# prompt = ChatPrompt(
#     "experimental/pirate",
#     name="pirate_prompt",    
# )

class LinkdinParagraph(BaseModel):
    idea: str
    paragraph: str

# prompt = ChatPrompt(
#     "linkdin_writer/body_paragraph", 
#     name="pirate_writer",  
#     pydantic_model=LinkdinParagraph
#     # prompt=exp_instruction.body_prompt,
# )

@prompt("linkdin_writer/body_paragraph", pydantic_model=LinkdinParagraph, rag_length=3)
async def pirate_writer(content):
    return content




In [ ]:
await pirate_writer(
    "write me a short paragraph about pirates",
    # name="pirate_prompt",
    topic="pirates",
    events=[],
    
    )

In [ ]:
from components.etl.chat_prompt import ChatResponse, ChatChunk

async for msg in pirate_writer.stream(
    "write me a short paragraph about pirates",
    topic="pirates",
    events=[]
    ):

    if type(msg) == ChatChunk:
        print(msg.content)
        print(msg.field)

In [ ]:
'{"id": "chatcmpl-91iIYrQjfSgWKu3ZnfQZiuQpIWq7n", "choices": [{"finish_reason": "stop", "index": 0, "logprobs": null, "message": {"content": "Idea: Shiver me timbers! Let\'s dive deep into the thrilling world of pirates and uncover some fascinating facts.\\nParagraph: Ahoy mateys! Did ye know that pirates weren\'t just a bunch of scallywags plundering the high seas for treasure? These swashbucklers actually had their own code of conduct called \\"The Pirate\'s Code\\" which set out rules for things like how to divvy up loot and how to settle disputes. And it\'s not all rum and sword fights, some pirates were skilled navigators and strategists, terrorizing the oceans with their cunning tactics. So next time you think of pirates, remember they were more than just eye patches and parrots, they were complex characters navigating a dangerous world.", "role": "assistant", "function_call": null, "tool_calls": null}}], "created": 1710195638, "model": "gpt-3.5-turbo-0125", "object": "chat.completion", "system_fingerprint": "fp_4f0b692a78", "usage": {"completion_tokens": 152, "prompt_tokens": 172, "total_tokens": 324}}'

In [ ]:
from components.etl.chat_prompt import ChatResponse, ChatChunk

async for msg in pirate_writer.stream(
    "write me a short paragraph about pirates",
    topic="pirates",
    events=[]
    ):

    if type(msg) == ChatChunk:
        print(msg.content)
        print(msg.field)

In [ ]:
msg.value.paragraph

In [ ]:
from components.etl.chat_prompt import ChatChunk, PromptChunkTypes
from components.etl.view_agent import AgentChunk, AgentChunkTypes

msg = AgentChunk(
    msg_type=AgentChunkTypes.AGENT_FINISH,
    func="unknown_task",
    prompt_chunk=ChatChunk(
        value="I don't know how to do this task",
        msg_type=PromptChunkTypes.PROMPT_FINISH,
    )
)


msg.json()